In [ ]:
from collections import defaultdict
import random
import os
import subprocess

import launchflow as lf

In [ ]:
random_int = random.randint(1, 1000)
environment_a = f"test-env-{random_int}"
environment_b = f"test-env-{random_int + 1}"
environments = [environment_a, environment_b]

In [ ]:
for env in environments:
    subprocess.check_call(
        f"lf environments create {env} --env-type=development --cloud-provider=gcp -y",
        shell=True,
        cwd=os.getcwd(),
    )

In [ ]:
environment_resources = defaultdict(list)
for env in environments:
    lf.environment = env
    environment_resources[env].append(lf.gcp.GCSBucket(f"notebook-integration-test-gcs-bucket-{env}"))
    environment_resources[env].append(lf.gcp.CloudRun("fastapi-service", dockerfile="Dockerfile"))

In [ ]:
results = []

for env in environments:
    lf.environment = env
    resources_to_create = environment_resources[env]
    results.append(await lf.create(*resources_to_create, environment=env, prompt=False, verbose=False))

assert not any(len(r.failed_plans) > 0 for r in results)

In [ ]:
results = []

for env in environments:
    lf.environment = env
    resources_to_destroy = environment_resources[env]
    results.append(await lf.destroy(env, prompt=False, verbose=False))

assert all(results)

In [ ]:
for env in environments:
    subprocess.check_call(
        f"lf environments delete {env} -y",
        shell=True,
        cwd=os.getcwd(),
    )